In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_histogram, plot_line, get_post_df
from codebase.ibis import post_process_sign, exp_and_normalise
import altair as alt
from codebase.classes_data import Data
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [3]:
log_dir1 = 'log/20210316_093129_mcmc_rasch_dense/'
exp_data1 = load_obj( 'complete_data', log_dir1)
ps_mcmc = load_obj('mcmc_post_samples', log_dir1)


In [24]:
# log_dir3 = 'log/20210302_203421_rash01/'
# log_dir3 = 'log/20210302_211048_rash01/'
# log_dir3 = 'log/20210303_062256_rash02/'
# log_dir3 = 'log/20210303_102221_rash03/'
# log_dir3 = 'log/20210305_103455_smc2_test/'
log_dir3 = 'log/20210316_160722__fabianrun//'
# exp_data3 = load_obj('data', log_dir3)
t = load_obj('t', log_dir3)
print(t)


21


In [5]:
particles = load_obj('particles', log_dir3)
ps_smc2 = dict()
for name in particles.param_names:
    ps_smc2[name] = particles.extract_particles_in_numpy_array(name)
particles.weights

array([-8.87418665, -2.50994956, -2.93639504, -3.96941627, -3.95621475,
       -6.25381591, -3.77388618, -4.36346147, -3.42854972, -3.30482329])

In [7]:
particles.get_acceptance_rate_for_particle_m(4).mean()

0.3833333333333333

In [8]:
particles.get_threshold_ess_indicator()[50:75].sum()

0

In [9]:
particles.get_threshold_ess_indicator()[:50].sum()

3

In [11]:
load_obj('jitter_corrs', log_dir3)

{0: {'beta': array([[-4.19870337e-17],
         [-4.19870337e-17],
         [-4.19870337e-17],
         [-4.19870337e-17],
         [-4.19870337e-17],
         [-4.19870337e-17]]),
  'alpha': array([-3.35549472e-16, -1.49829475e-16, -7.16904175e-17, -1.91522934e-16,
                     nan,  1.65906017e-16])},
 1: {'beta': array([[-0.62760076],
         [-0.62760076],
         [-0.62760076],
         [-0.62760076],
         [-0.62760076],
         [-0.62760076]]),
  'alpha': array([-0.12000265, -0.021081  ,  0.23557098, -0.24871203,  0.13187983,
         -0.25866396])},
 2: {'beta': array([[nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan]]),
  'alpha': array([            nan,  1.15055591e-17,  4.46008216e-16, -5.27031208e-17,
                     nan,             nan])},
 3: {'beta': array([[nan],
         [nan],
         [nan],
         [nan],
         [nan],
         [nan]]),
  'alpha': array([7.87973347e-17,            nan,            nan, 0.0000

In [12]:
# particles = load_obj('particles', log_dir3)
particles.resample_particles_bundles()
# ps_smc2 = dict()
# for name in particles.param_names:
#     ps_smc2[name] = particles.extract_particles_in_numpy_array(name)


##

## Or load existing directory

In [13]:
ps_mcmc = post_process_sign(ps_mcmc)
ps_smc2 = post_process_sign(ps_smc2)


## Plot MCMC samples

In [14]:
param = 'beta'
df = get_post_df(ps_mcmc[param]) 
df = df[df.row==0]
df['source'] = 'mcmc'


df3 = get_post_df(ps_smc2[param]) 
df3 = df3[df3.row==0]
df3['source'] = 'smc2'

plot_density(pd.concat([df, df3]), height=100) 

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

In [15]:
param = 'alpha'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'


df3 = get_post_df(ps_smc2[param]) 
df3['source'] = 'smc2'

plot_density(pd.concat([df, df3]), height=100)

alt.Chart(...)

## Plot quantiles

In [16]:
param = 'beta'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df[df.row==0]).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

## chart overlaying quantiles with real data

In [17]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [18]:
plot_data = plot_data[plot_data['index']=='r_0.c_0']

c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)


## chart overlaying quantiles with real data

In [19]:
param = 'alpha'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

In [20]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['alpha'], columns=['data'])
dd['col'] = np.arange(6)
dd['row'] = 0
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [21]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-5,5])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)